# PA Opioid Epidemic

In [253]:
import pandas as pd
import numpy as np
import matplotlib as plt
import folium

In [254]:
arrest_dat = 'Opioid_Seizures_and_Arrests_CY_2013_-_Current_Quarterly_County_State_Police.csv'
overdose_dat = 'Overdose_Information_Network_Data_CY_January_2018_-_Current_Monthly_County_State_Police.csv'
county_geo = 'PaCounty2020_01.geojson'

In [255]:
arr_df = pd.read_csv(arrest_dat)
ovd_df = pd.read_csv(overdose_dat)

In [256]:
odf = ovd_df.drop(['Response Time Desc', 'Revive Action Desc', 'Response Desc', 'Victim County', 'Victim OD Drug ID', 'Third Party Admin Desc', 'Administration ID', 'Dose Count', 'Dose Unit', 'Dose Desc', 'Ethnicity Desc', 'Incident ID', 'Victim State', 'Incident State', 'Victim County Latitude and Longitude', 'Incident County Latitude and Longitude', 'Victim ID', 'Victim County FIPS Code', 'Incident County FIPS Code', 'Victim County Latitude', 'Victim County Longitude'], axis=1)
odf = odf.rename(columns={"Accidental Exposure": "Accident", "Age Range": "Age", "Naloxone Administered": "Naloxone", "Incident County Longitude": "Longitude", "Incident County Latitude": "Latitude", "Susp OD Drug Desc": "Drug", "Gender Desc": "Gender", "Incident Date": "Date", "Incident Time": "Time", "Incident County Name": "County"})
odf['Date'] = odf['Date'] + ":" + odf['Time']
odf = odf.drop(['Time'], axis=1)

In [257]:
odf['Date'] = odf['Date'].replace(to_replace=r'\/', value=':', regex=True)
odf['Date'] = pd.to_datetime(odf['Date'], format='%m:%d:%Y:%H:%M:%S')
odf['Day'] = odf['Day'].astype('category')

In [258]:
odf['Age'] = odf['Age'].replace(to_replace=r'\s\-\s', value='-', regex=True)
odf['Age'] = odf['Age'].astype('category')

In [259]:
odf['Gender'] = odf['Gender'].replace(to_replace=r'Male', value='0', regex=True)
odf['Gender'] = odf['Gender'].replace(to_replace=r'Female', value='1', regex=True)
odf['Gender'] = odf['Gender'].replace(to_replace=r'Unknown', value='2', regex=True)
odf['Gender'] = odf['Gender'].astype(int)

In [260]:
odf['Accident'] = odf['Accident'].replace(to_replace=r'N', value='0', regex=True)
odf['Accident'] = odf['Accident'].replace(to_replace=r'Y', value='1', regex=True)
odf['Accident'] = odf['Accident'].astype(int)

In [261]:
odf['Naloxone'] = odf['Naloxone'].replace(to_replace=r'N', value='0', regex=True)
odf['Naloxone'] = odf['Naloxone'].replace(to_replace=r'Y', value='1', regex=True)
odf['Naloxone'] = odf['Naloxone'].astype(int)

In [262]:
odf['Survive'] = odf['Survive'].replace(to_replace=r'N', value='0', regex=True)
odf['Survive'] = odf['Survive'].replace(to_replace=r'Y', value='1', regex=True)
odf['Survive'] = odf['Survive'].replace(to_replace=r'U', value='2', regex=True)
odf['Survive'] = odf['Survive'].astype(int)

In [263]:
odf['Race'] = odf['Race'].astype('category')

In [264]:
odf['County'] = odf['County'].astype('category')

In [265]:
odf['Drug'] = odf['Drug'].astype('category')

In [266]:
odf.head()

,Date,Day,County,Gender,Age,Race,Accident,Drug,Naloxone,Survive,Latitude,Longitude
0,2020-07-02 12:48:00,Thursday,York,0,50-59,White,0,HEROIN,1,1,39.921925,-76.725761
1,2020-09-20 23:11:00,Sunday,York,0,20-24,White,0,FENTANYL ANALOG/OTHER SYNTHETIC OPIOID,1,1,39.921925,-76.725761
2,2020-09-11 20:00:00,Friday,Union,0,50-59,White,0,HEROIN,0,0,40.964731,-77.064158
3,2020-07-10 21:10:00,Friday,York,0,40-49,White,0,FENTANYL,1,1,39.921925,-76.725761
4,2020-06-20 20:52:00,Saturday,York,0,40-49,White,0,FENTANYL,0,0,39.921925,-76.725761


In [267]:
arr_df.shape

(2290, 16)

In [268]:
arr_df.head()

,Year,Qtr,Qtr Start Date,County Code,County Code Text,FIPS State Code,FIPS County Code,County Name,Drug,Incident Count,Drug Quantity,Unit Used,Arrests,Latitude,Longitude,Georeferenced Latitude & Longitude
0,2020,Q3,07/01/2020,63,63,42,125,Washington,Heroin,19,0.02982,Kilograms,30,40.191097,-80.251801,POINT (-80.251801 40.191097)
1,2015,Q4,10/01/2015,5,5,42,9,Bedford,Heroin,3,0.00480,Kilograms,2,40.007375,-78.491165,POINT (-78.491165 40.007375)
2,2016,Q3,07/01/2016,59,59,42,117,Tioga,Heroin,1,0.02830,Kilograms,2,41.773338,-77.257881,POINT (-77.257881 41.773338)
3,2013,Q2,04/01/2013,22,22,42,43,Dauphin,Heroin,3,0.00133,Kilograms,1,40.419746,-76.779606,POINT (-76.779606 40.419746)
4,2020,Q4,10/01/2020,18,18,42,35,Clinton,Fentanyl,1,0.01100,Kilograms,1,41.232863,-77.642838,POINT (-77.642838 41.232863)


In [269]:
adf = arr_df.drop(['Unit Used', 'Year', 'Qtr', 'Qtr Start Date', 'County Code', 'County Code Text', 'FIPS County Code', 'FIPS State Code', 'Georeferenced Latitude & Longitude'], axis=1)
adf = adf.rename(columns={"Drug Quantity": "kg", "County Name": "County", "Incident Count": "Incidents"})

In [270]:
adf.shape

(2290, 7)